In [13]:
import os
import requests
import subprocess
import geopandas as gpd
import pandas as pd

In [14]:
today = pd.Timestamp.today().strftime('%Y-%m-%d')

In [ ]:
# Requires local https://github.com/openaddresses/pyesridump install

In [ ]:
# Define the URL and output file path
layer_url = "https://services2.arcgis.com/zNjnZafDYCAJAbN0/ArcGIS/rest/services/Street_ROW_Trees/FeatureServer/0"
output_file = "data/raw/street_trees.geojson"

# Create the directory if it doesn't exist
os.makedirs("../data/raw/cities", exist_ok=True)

# Use subprocess to call esri2geojson
subprocess.run(["esri2geojson", layer_url, output_file])

# Read the GeoJSON file into a GeoDataFrame
street_trees_gdf = gpd.read_file(output_file)

# Display the dataframe
street_trees_gdf.head()

In [17]:
len(street_trees_gdf)

57070

In [22]:
street_trees_gdf['fetched_on'] = today
street_trees_gdf['municipality'] = 'Pasadena'
street_trees_gdf['source'] = 'Esri endpoint'

In [ ]:
street_trees_gdf.columns = street_trees_gdf.columns.str.lower()

In [37]:
street_trees_gdf.columns

Index(['objectid', 'tree_rec', 'alt_tree_id', 'common_name', 'genus',
       'species', 'status_text', 'address', 'street_direction', 'street_name',
       'street_type', 'street_suffix', 'trunk_dia', 'x_coordinate',
       'y_coordinate', 'classification_text', 'geometry', 'fetched_on',
       'municipality', 'source', 'lon', 'lat'],
      dtype='object')

In [ ]:
street_trees_gdf['lon'] = street_trees_gdf.geometry.x
street_trees_gdf['lat'] = street_trees_gdf.geometry.y

In [41]:
street_trees_gdf['address'] = street_trees_gdf['address'].astype(str).str.replace('.0', '')

In [43]:
street_trees_gdf[['tree_rec', 'common_name', 'genus',
       'species', 'status_text', 'address', 'street_direction', 'street_name',
       'street_type', 'trunk_dia', 'fetched_on',
       'municipality', 'lon', 'lat']].to_csv('../data/processed/cities/pasadena.csv', index=False)